In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
sns.set_style("ticks")
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
import shutil
from os import listdir
from os.path import join, isfile
import datetime as dt

from pathlib import Path

#from google.colab import drive
#
# drive.mount('/content/drive')


In [2]:
os.getcwd()

'/Users/mhausch/Data/Prod_Dashboards/Source'

In [3]:
#Define the pqth to the data directory
dir_path = "../Data"

#File names for import
raw_sfdc = "latest salesforce.xlsx"
country_list = "lookup (1).xlsx"
latest_revenue = "latest_revenue.xlsx"

In [4]:
#create the file path 
path_sdfc = Path(dir_path, raw_sfdc)
path_country_list = Path(dir_path, country_list)
path_revenue = Path(dir_path, latest_revenue)


In [5]:
#Import different datasources
new_df = pd.read_excel(path_sdfc, sheet_name = 0,skiprows=10)
lookup_countries = pd.read_excel(path_country_list, sheet_name = 0)
lookup_type = pd.read_excel(path_country_list, sheet_name = "Type")
lookup_cancer_type = pd.read_excel(path_country_list, sheet_name = "Cancer_type")
lookup_currency = pd.read_excel(path_country_list, sheet_name = "Currency", parse_dates= ["Month"]).set_index("Month")
lookup_currency_type =  pd.read_excel(path_country_list, sheet_name = "curr_look")
lookup_cancer_type = pd.read_excel(path_country_list, sheet_name = "Cancer_type")
product_dict= pd.read_excel(path_country_list, sheet_name = "product list")
df = pd.read_excel(path_revenue, sheet_name = 0, skiprows=28)

In [6]:
#Import the data from the different data sources
country_dict = dict(zip(lookup_countries["Reporting_Country_short"], lookup_countries["Account_Reporting_Country"]))
AE_dict = { 'CANADA': "AE", "SPAIN" : "AE", "UNITED KINGDOM" : "AE", "BRAZIL" : "AE", "TURKEY" : "AE", "IRELAND" : "AE"}
subregion_dict = dict(zip(lookup_countries["Account_Reporting_Country"], lookup_countries["Sub_region"]))
lookup_currency["EUR"] = lookup_currency["EUR"]/1000000
lookup_currency["GBP"] = lookup_currency["GBP"]/1000000
lookup_currency["USD"] = 1
euaa_country_list = list(country_dict.values())
curr_dict = dict(zip(lookup_currency_type["Account_Reporting_Country"], lookup_currency_type["Currency"]))
type_dict = dict(zip(lookup_type["LIMS Project ID"], lookup_type["Type"]))
cancer_type_dict = dict(zip(lookup_cancer_type["Old"], lookup_cancer_type["New"]))
unique_LIMS_id = list(set(new_df["LIMS Project ID"]))
unique_lookup_id = list(set(lookup_type["LIMS Project ID"]))
product_dict = dict(zip(product_dict["lims_id"], product_dict["test"]))

In [7]:
new_df["Reporting Country  ↑"].fillna(method='ffill',inplace=True)
new_df["Reporting Country"] = new_df["Reporting Country  ↑"].map(country_dict)
new_df["Type"] = new_df["LIMS Project ID"].map(type_dict)
new_df["Cancer Type_mapped"] = new_df["Cancer Type"].map(cancer_type_dict)
def merge_ISTs(x):
  if x == "GH Trials (Prospective)":
    return "ISTs"
  elif x == "Lunar":
    return "ISTs"
  else:
    return x
new_df["Type"] = new_df["Type"].map(merge_ISTs)

In [8]:
new_df.tail(2)

,Unnamed: 0,Reporting Country ↑,Unnamed: 2,Date Received,Accession #,Request Type,Account: Account Name,Contact: Full Name,Test Ordered,Sample Gene Panel,LIMS Project ID,Billing Type,Cancer Type,Final Report Date,Latest Report Date,Reporting Country,Type,Cancer Type_mapped
20992,NaN,Confidential Information - Do Not Distribute,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20993,NaN,"Copyright © 2000-2023 salesforce.com, inc. All...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
new_df["Accession #"] = new_df["Accession #"].fillna("Unknown")
new_df["Reporting Country_mapped"] = new_df["Reporting Country"]
new_df["Product"] = 1
new_df["AE_supported"] = np.where(new_df["Reporting Country"].isin(AE_dict.keys()), "AE", "non_AE")
new_df["Currency"] = new_df["Reporting Country_mapped"].map(curr_dict)
new_df["Sub_region"] =new_df["Reporting Country_mapped"]
new_df["Sub_region"] = new_df["Sub_region"].map(subregion_dict)
new_df= new_df[new_df["Reporting Country"].isin(euaa_country_list)].copy()

In [10]:
new_df["LIMS Project ID"].value_counts()

LIMS Project ID
AZN_96       4862
GHI_01       3298
GHI_19       2157
PGS_01        694
CUB_01        643
             ... 
FOR_01_EU       2
UNI_02          1
PFE_52_EU       1
GHI_05          1
AMG_06ROW       1
Name: count, Length: 81, dtype: int64

In [11]:
new_df["Date Received"] = pd.to_datetime(new_df["Date Received"])
new_df["Date Received_index"]=new_df["Date Received"]
new_df = new_df.set_index("Date Received_index")
new_df["Month"] = new_df.index.month
new_df["Year"] = new_df.index.year
new_df['Quarter'] = new_df.index.quarter
new_df['Sales'] = 1
new_df['Year_Month'] = new_df.index.to_period('M')
new_df['Year_Month'] = new_df['Year_Month'].astype(str)
new_df['Year_Month'] = pd.to_datetime(new_df['Year_Month'])
new_df["Type"] = new_df['Type'].astype('category')
new_df["Month_to_match"] = new_df["Year"].astype("string") + "-" + new_df["Month"].astype("string")
new_df.drop(["Unnamed: 0", "Unnamed: 2"], axis = 1)

,Reporting Country ↑,Date Received,Accession #,Request Type,Account: Account Name,Contact: Full Name,Test Ordered,Sample Gene Panel,LIMS Project ID,Billing Type,Cancer Type,Final Report Date,Latest Report Date,Reporting Country,Type,Cancer Type_mapped,Reporting Country_mapped,Product,AE_supported,Currency,Sub_region,Month,Year,Quarter,Sales,Year_Month,Month_to_match
Date Received_index,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-04-28,AR,2020-04-28,A0251277,Pharma,RHI_E1: Centro Medico Austral,Guillermo Striech,NaN,2.11,RHI_E1,Pharma : ROW,Breast Carcinoma,5/3/2020,5/3/2020,ARGENTINA,Biopharma,Breast,ARGENTINA,1,non_AE,USD,Americas,4,2020,2,1,2020-04-01,2020-4
2020-06-05,AR,2020-06-05,A0261162,Pharma,RHI_E1: Centro Medico Austral,Guillermo Striech,NaN,2.11,RHI_E1,Pharma : ROW,Breast Carcinoma,6/11/2020,6/11/2020,ARGENTINA,Biopharma,Breast,ARGENTINA,1,non_AE,USD,Americas,6,2020,2,1,2020-06-01,2020-6
2021-06-04,AR,2021-06-04,A0362649,Pharma,JNJ_20: Clinica Viedma,Ruben Kowalyszyn,Guardant 360,2.11,JNJ_20,Do Not Bill - Cancelled,Lung adenocarcinoma,6/9/2021,6/9/2021,ARGENTINA,Biopharma,NaN,ARGENTINA,1,non_AE,USD,Americas,6,2021,2,1,2021-06-01,2021-6
2021-11-12,AR,2021-11-12,A0427581,Pharma,JNJ_20: Centro Oncologico Koren,Ernesto Korbenfeld,Guardant 360,2.11,JNJ_20,Pharma : ROW,Lung adenocarcinoma,11/19/2021,11/19/2021,ARGENTINA,Biopharma,NaN,ARGENTINA,1,non_AE,USD,Americas,11,2021,4,1,2021-11-01,2021-11
2023-09-22,AR,2023-09-22,A0856826,Pharma,NVS_TO1: Hospital Privado de Cordoba,Jeremias Sierra,NaN,Default,NVS_TO1,Pharma : ROW,Non-small cell lung carcinoma (NSCLC),9/27/2023,9/27/2023,ARGENTINA,NaN,NaN,ARGENTINA,1,non_AE,USD,Americas,9,2023,3,1,2023-09-01,2023-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-21,UY,2023-08-21,A0832101,Clinical,Practice Dr Luis Ubillos,Luis Ubillos,Guardant 360,2.11,GHI_01,Self-Pay Patient,Ovarian carcinoma,8/25/2023,8/25/2023,URUGUAY,Commercial,NaN,URUGUAY,1,non_AE,USD,Americas,8,2023,3,1,2023-08-01,2023-8
2023-09-25,UY,2023-09-25,A0858523,Clinical,Practice Dr Luis Ubillos,Luis Ubillos,83-Gene Panel,2.12,GH2_12,Self-Pay Patient,Prostate adenocarcinoma,10/2/2023,10/2/2023,URUGUAY,Commercial,NaN,URUGUAY,1,non_AE,USD,Americas,9,2023,3,1,2023-09-01,2023-9
2023-03-20,UY,2023-03-20,A0690872,Clinical,Practice Dr Luis Ubillos,Luis Ubillos,Guardant 360,2.11,GHI_01,Self-Pay Patient,Breast Carcinoma,3/24/2023,3/24/2023,URUGUAY,Commercial,Breast,URUGUAY,1,non_AE,USD,Americas,3,2023,1,1,2023-03-01,2023-3


In [12]:

#product_dict = {"GHI_10":"Reveal", "GHI_18": "Reveal", "GHI_19" : "Guardant360 CDx", "GHI_01": "Guardant360 CDx", "GH2_12" : "G360/2.12","GH_Reveal_3_0":"Reveal" }
new_df["new LIMS Project ID"]=new_df["LIMS Project ID"].map(product_dict)
new_df["Quarter_Year"]=new_df.index.to_period('Q')
new_df["Quarter_Year"]=new_df["Quarter_Year"].astype(str)
# subregion erweitern und hinzufügen
new_subregion_dict={'CANADA': 'Canada',
 'SPAIN': 'Europe',
 'ITALY': 'Europe',
 'URUGUAY': 'South America',
 'BRAZIL': 'South America',
 'ARGENTINA': 'South America',
 'HUNGARY': 'Europe',
 'UNITED KINGDOM': 'Europe',
 'MEXICO': 'South America',
 'BELGIUM': 'Europe',
 'TURKEY': 'Europe',
 'COLOMBIA': 'South America',
 'PERU': 'South America',
 'FRANCE': 'Europe',
 'GERMANY': 'Europe',
 'PORTUGAL': 'Europe',
 'SWITZERLAND': 'Europe',
 'COSTA RICA': 'South America',
 'CHILE': 'South America',
 'SWEDEN': 'Europe',
 'NETHERLANDS': 'Europe',
 'DENMARK': 'Europe',
 'IRELAND': 'Europe',
 'GREECE': 'Europe',
 'AUSTRIA': 'Europe',
 'CZECH REPUBLIC': 'Europe',
 'CROATIA': 'Europe',
 'UKRAINE': 'Europe',
 'SLOVENIA': 'Europe',
 'RUSSIA': 'Europe',
 'BELARUS': 'Europe',
 'FINLAND': 'Europe',
 'PUERTO RICO': 'South America',
 'LITHUANIA': 'Europe',
 'NORTH MACEDONIA': 'Europe',
 'ROMANIA': 'Europe',
 'LUXEMBOURG': 'Europe',
 'POLAND': 'Europe'}
new_df["adv_Sub_region"]= new_df ["Reporting Country"]
new_df["adv_Sub_region"] = new_df["adv_Sub_region"].map(new_subregion_dict)

In [13]:
new_df.index.min(), new_df.index.max()

(Timestamp('2020-01-02 00:00:00'), Timestamp('2023-11-29 00:00:00'))

In [14]:
df['Accession #']= df['Accession #'].fillna("missing")
df["Account: Reporting Country  ↑"].fillna(method='ffill',inplace=True)
df = df[df['Accession #'].str.startswith("A0")]
gf=df.copy()
only_string=df[df["Accession #"].str.contains("/")==True]
first_try=only_string.copy()
first_try["Accession split"]=first_try["Accession #"].str.split("/").str[1]
first_try["Accession split"]=first_try["Accession split"].str.strip() #
testas=new_df["Accession #"].unique()
first_finished=first_try[first_try["Accession split"].isin(testas)]
fertige=first_finished["Accession #"].unique()
second_finished=only_string[only_string["Accession #"].isin(fertige)==False]
second_finished["Accession split"]=second_finished["Accession #"].str.split("/").str[0]
second_finished["Accession split"]=second_finished["Accession split"].str.strip()


In [15]:
first_finished_all = pd.concat([first_finished,second_finished] )
first_finished_all["Accession #"]=first_finished_all["Accession split"]


In [16]:
df= pd.concat([df,first_finished_all])

In [17]:
df.tail()

,2019-09-01 00:00:00,Account: Reporting Country ↑,Date Received,Accession #,Account: Account Name,Contact: Full Name,Billing Type,Final Report Date,Invoiced Ammount,Expected Amount $,Invoiced/In Prg.,FOC,Collected,Unnamed: 13,Unnamed: 14,Unnamed: 15,Total,1943954.409,Accession split
6586,NaN,GB,2023-11-24 00:00:00,A0908580,EDX Medical Ltd,Hendrik-Tobias Arkenau,Self-Pay Patient,2023-11-29 00:00:00,3490 GBP,EDx Sample??? - Emailed Erik to confirm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A0908580
6714,NaN,GB,2023-11-01 00:00:00,A0889083,Wellington Hospital,Martyn Caplin,Self-Pay Patient,2023-11-08 00:00:00,2240 GBP,NaN,NaN,NaN,y (BUPA),NaN,NaN,NaN,NaN,NaN,A0889083
6747,NaN,TR,2023-11-24 00:00:00,A0907362,Valentis Saglik Hizmetleri ve Danismanlik AS,Haci Ibrahim Petekaya,Distributors : ROW,NaN,1838,Accounts Receivable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A0907362
4917,NaN,GB,2023-03-30 00:00:00,A0698831,Clatterbridge Cancer Centre,Carlo Palmieri,Do Not Bill - Cancelled,NaN,0 GBP,Cancelled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A0698831
6291,NaN,GB,2023-09-01 00:00:00,A0840806,HCA Laboratories,Siow-Ming Lee,Distributors : ROW,2023-09-15 00:00:00,2178.75 GBP,Accounts Receivable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A0840806


In [18]:

df=df[df["Accession #"].str.contains("/")==False]
df["Accession #"]=df["Accession #"].str[:8]
df["Date Received"] = pd.to_datetime(df["Date Received"])
df= df.set_index("Date Received")
revenue_col_list = ['Accession #', 'Invoiced Ammount','Expected Amount $','Invoiced/In Prg.', 'FOC']
df["FOC"] = df["FOC"].fillna("")
def foc_clean_up(value):
    value=value.strip()
    if len(value)!=0:
        return "Free"
    else:
        return "Paid"
df["FOC"] = df["FOC"].apply(foc_clean_up).astype("category")
rev_df = df.loc[:, revenue_col_list].copy()

In [19]:
df_merged = new_df.merge(rev_df[["Accession #", "FOC", "Invoiced Ammount"]], how='inner', on = "Accession #")
eff=df_merged.copy()
len(eff),len(df)


(5839, 6075)

In [20]:
eff["Invoiced Ammount"] = eff["Invoiced Ammount"].astype("string")
eff["Revenue(Local_Curr)"] = eff["Invoiced Ammount"].str.extract('(\d+,?\d*)')
eff["Revenue(Local_Curr)"] = eff["Revenue(Local_Curr)"].fillna("0")
eff["Revenue(Local_Curr)"] = eff["Revenue(Local_Curr)"].str.replace(",","")
eff["Revenue(Local_Curr)"] = eff["Revenue(Local_Curr)"].astype(float)
lookup_currency["Date"] = lookup_currency.index.to_numpy().astype('datetime64[M]').astype(str)
lookup_currency_wide=lookup_currency
lookup_currency_long = lookup_currency_wide.melt(id_vars = ["Date"], value_vars = ["EUR", "GBP", "USD"], var_name = "Currency", value_name = "Value")


In [21]:
eff["Month_to_match"] = eff["Year_Month"].dt.to_period('M').astype("string")
revenue= eff.merge(lookup_currency_long, left_on = ["Month_to_match", "Currency"], right_on = ["Date", "Currency"])
revenue["Revenue (USD)"] = revenue["Revenue(Local_Curr)"]*revenue["Value"]
# set index for revenue so that its sorted
revenue["Date Received_index"]=revenue["Date Received"]
revenue = revenue.set_index("Date Received_index")
revenue.sort_index(inplace=True)
def set_segments_old(price):
    if price<1000:
        return "0-1000$"
    elif price<=1250:
        return "1000-1250$"
    elif price<=1500:
        return"1250-1500$ "
    elif price<=1750:
        return"1250-1750$"
    elif price<=2000:
        return"1750-2000$"
    elif price<=2250:
        return"2000-2250$"
    elif price<=2500:
        return"2250-2500$"
    elif price<=2750:
        return"2500-2750$"
    elif price<=3000:
        return"2750-3000+ $"
    else:
        return "3000+"
#revenue["price_segment"]= revenue["Revenue (USD)"].map(set_segments_old)

revenue = revenue.drop(["Unnamed: 0", "Unnamed: 2"], axis = 1)

In [22]:
revenue.head(2)

,Reporting Country ↑,Date Received,Accession #,Request Type,Account: Account Name,Contact: Full Name,Test Ordered,Sample Gene Panel,LIMS Project ID,Billing Type,Cancer Type,Final Report Date,Latest Report Date,Reporting Country,Type,Cancer Type_mapped,Reporting Country_mapped,Product,AE_supported,Currency,Sub_region,Month,Year,Quarter,Sales,Year_Month,Month_to_match,new LIMS Project ID,Quarter_Year,adv_Sub_region,FOC,Invoiced Ammount,Revenue(Local_Curr),Date,Value,Revenue (USD)
Date Received_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-02,ES,2020-01-02,A0223879,Clinical,Hospital Quiron Barcelona,Jesus Soberino Garcia,Guardant 360,2.11,GHI_01,Distributors : ROW,Breast Carcinoma,1/7/2020,9/28/2020,SPAIN,Commercial,Breast,SPAIN,1,AE,EUR,Europe,1,2020,1,1,2020-01-01,2020-01,Guardant360 CDx,2020Q1,Europe,Paid,2303,2303.0,2020-01,1.11,2555.98
2020-01-02,GB,2020-01-02,A0223865,Clinical,The Nuffield Hospital Guildford,Ajay Mehta,Guardant 360,2.11,GHI_01,Self-Pay Patient,Non-small cell lung carcinoma (NSCLC),1/14/2020,9/28/2020,UNITED KINGDOM,Commercial,NaN,UNITED KINGDOM,1,AE,GBP,Europe,1,2020,1,1,2020-01-01,2020-01,Guardant360 CDx,2020Q1,Europe,Paid,"GBP 2,490.00",2490.0,2020-01,1.31,3255.64


In [23]:

def set_segments(price):
    if price==0:
        return "0 $"
    elif price<=1500:
        return "0-1500$ "
    elif price<=2000:
        return"1500-2000$ "
    elif price<=3000:
        return"2000-3000$ "
    else:
        return"3000+ $"
        #filter out duplicates
revenue["price_segment"]= revenue["Revenue (USD)"].map(set_segments)
revenue= revenue.drop_duplicates(subset="Accession #")
new_df=  new_df.drop_duplicates(subset="Accession #")
new_df = new_df.drop(["Unnamed: 0", "Unnamed: 2"], axis = 1)


In [24]:
new_df.head()
new_df["LIMS Project ID"].value_counts()
len(new_df)

20916

In [25]:
revenue.tail()

,Reporting Country ↑,Date Received,Accession #,Request Type,Account: Account Name,Contact: Full Name,Test Ordered,Sample Gene Panel,LIMS Project ID,Billing Type,Cancer Type,Final Report Date,Latest Report Date,Reporting Country,Type,Cancer Type_mapped,Reporting Country_mapped,Product,AE_supported,Currency,Sub_region,Month,Year,Quarter,Sales,Year_Month,Month_to_match,new LIMS Project ID,Quarter_Year,adv_Sub_region,FOC,Invoiced Ammount,Revenue(Local_Curr),Date,Value,Revenue (USD),price_segment
Date Received_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-11-28,TR,2023-11-28,A0907934,Clinical,Valentis Saglik Hizmetleri ve Danismanlik AS,Ibrahim Yildiz,83-Gene Panel,2.12,GH2_12,Distributors : ROW,Prostate adenocarcinoma,NaN,11/28/2023,TURKEY,Commercial,NaN,TURKEY,1,AE,USD,Europe,11,2023,4,1,2023-11-01,2023-11,G360/2.12,2023Q4,Europe,Paid,2319,2319.0,2023-11,1.00,2319.0,2000-3000$
2023-11-29,GB,2023-11-29,A0908992,Clinical,Kent Oncology Centre,Timothy Sevitt,Guardant 360,NaN,GHI_19,Self-Pay Patient,Pending,NaN,11/29/2023,UNITED KINGDOM,Commercial,Other,UNITED KINGDOM,1,AE,GBP,Europe,11,2023,4,1,2023-11-01,2023-11,Guardant360 CDx,2023Q4,Europe,Paid,<NA>,0.0,2023-11,1.18,0.0,0 $
2023-11-29,ES,2023-11-29,A0909001,Clinical,Hospital Clinico Provincial de Barcelona,Noemi Reguart,Guardant 360,NaN,GHI_19,Distributors : ROW,Lung adenocarcinoma,NaN,11/29/2023,SPAIN,Commercial,NaN,SPAIN,1,AE,EUR,Europe,11,2023,4,1,2023-11-01,2023-11,Guardant360 CDx,2023Q4,Europe,Paid,<NA>,0.0,2023-11,1.00,0.0,0 $
2023-11-29,GB,2023-11-29,A0909135,Clinical,Guys Hospital,Jose Roca,Guardant 360,2.12,GH2_12,Self-Pay Patient,Pending,NaN,11/29/2023,UNITED KINGDOM,Commercial,Other,UNITED KINGDOM,1,AE,GBP,Europe,11,2023,4,1,2023-11-01,2023-11,G360/2.12,2023Q4,Europe,Paid,<NA>,0.0,2023-11,1.18,0.0,0 $
2023-11-29,TR,2023-11-29,A0908934,Clinical,Valentis Saglik Hizmetleri ve Danismanlik AS,Mehmet Akif Ozturk,83-Gene Panel,2.12,GH2_12,Distributors : ROW,Small cell lung carcinoma,NaN,11/29/2023,TURKEY,Commercial,NaN,TURKEY,1,AE,USD,Europe,11,2023,4,1,2023-11-01,2023-11,G360/2.12,2023Q4,Europe,Paid,<NA>,0.0,2023-11,1.00,0.0,0 $


In [26]:
# save the file
#path = Path(dir_path, "revenue_only.csv")
#'/content/drive/MyDrive/Colab Notebooks/revenue_only.csv'
#with open(path, 'w', encoding = 'utf-8-sig') as f:
#revenue.to_csv(f)
#path = Path(dir_path, 'all_data.csv')
#with open(path, 'w', encoding = 'utf-8-sig') as f:
  #new_df.to_csv(f)

In [27]:
revenue.index.max(), revenue.index.min()

(Timestamp('2023-11-29 00:00:00'), Timestamp('2020-01-02 00:00:00'))

In [28]:
new_df["new LIMS Project ID"].isna()

Date Received_index
2020-04-28     True
2020-06-05     True
2021-06-04     True
2021-11-12     True
2023-09-22     True
              ...  
2023-08-21    False
2023-09-25    False
2023-03-20    False
2023-02-06    False
2023-07-26    False
Name: new LIMS Project ID, Length: 20916, dtype: bool

In [29]:
#new_df = new_df[new_df["new LIMS Project ID"].isna()]
#new_df = new_df.sort_index()

In [30]:
new_df["LIMS Project ID"].value_counts()

LIMS Project ID
AZN_96       4862
GHI_01       3298
GHI_19       2157
PGS_01        694
CUB_01        643
             ... 
FOR_01_EU       2
UNI_02          1
PFE_52_EU       1
GHI_05          1
AMG_06ROW       1
Name: count, Length: 81, dtype: int64

In [31]:
path_all_csv = Path(dir_path, "all_data.csv")
path_revenue_csv = Path(dir_path, "revenue_only.csv")

In [32]:
new_df.to_csv(path_all_csv, index= False)
revenue.to_csv(path_revenue_csv, index=False)

### Testing

In [33]:
#new_df = new_df.drop(["Unnamed: 0", "Unnamed: 2"], axis = 1)
new_df.sample(5)
new_df.index.max()

Timestamp('2023-11-29 00:00:00')

In [34]:
new_df["LIMS Project ID"].value_counts()

LIMS Project ID
AZN_96       4862
GHI_01       3298
GHI_19       2157
PGS_01        694
CUB_01        643
             ... 
FOR_01_EU       2
UNI_02          1
PFE_52_EU       1
GHI_05          1
AMG_06ROW       1
Name: count, Length: 81, dtype: int64

In [35]:
new_df.index.max()

Timestamp('2023-11-29 00:00:00')

In [36]:
new_df.sort_index().tail(5)

,Reporting Country ↑,Date Received,Accession #,Request Type,Account: Account Name,Contact: Full Name,Test Ordered,Sample Gene Panel,LIMS Project ID,Billing Type,Cancer Type,Final Report Date,Latest Report Date,Reporting Country,Type,Cancer Type_mapped,Reporting Country_mapped,Product,AE_supported,Currency,Sub_region,Month,Year,Quarter,Sales,Year_Month,Month_to_match,new LIMS Project ID,Quarter_Year,adv_Sub_region
Date Received_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-11-29,ES,2023-11-29,A0908621,Research,ORACLE: Hospital Universitario HMN Sanchinarro,Antonio Cubillo,NaN,LUNAR 1.0,ORACLE,Pharma : ROW,Breast Carcinoma,NaN,11/29/2023,SPAIN,ISTs,Breast,SPAIN,1,AE,EUR,Europe,11,2023,4,1,2023-11-01,2023-11,NaN,2023Q4,Europe
2023-11-29,IT,2023-11-29,A0909107,Pharma,AZN_96: Istituto Clinico Humanitas,Armando Santoro,NaN,2.11,AZN_96,Pharma : ROW,Breast Carcinoma,NaN,11/29/2023,ITALY,Biopharma,Breast,ITALY,1,non_AE,EUR,Europe,11,2023,4,1,2023-11-01,2023-11,NaN,2023Q4,Europe
2023-11-29,DE,2023-11-29,A0909102,Pharma,AZN_96: Rotkreuzklinik Muenchen,Michael Braun,NaN,2.11,AZN_96,Pharma : ROW,Breast Carcinoma,NaN,11/29/2023,GERMANY,Biopharma,Breast,GERMANY,1,non_AE,EUR,Europe,11,2023,4,1,2023-11-01,2023-11,NaN,2023Q4,Europe
2023-11-29,ES,2023-11-29,A0909001,Clinical,Hospital Clinico Provincial de Barcelona,Noemi Reguart,Guardant 360,NaN,GHI_19,Distributors : ROW,Lung adenocarcinoma,NaN,11/29/2023,SPAIN,Commercial,NaN,SPAIN,1,AE,EUR,Europe,11,2023,4,1,2023-11-01,2023-11,Guardant360 CDx,2023Q4,Europe
2023-11-29,GB,2023-11-29,A0908610,Pharma,TRACC STUDY: Royal Shrewsbury Hospital,Yash Choudhary,NaN,Default,TRCC_03,Pharma : ROW,Colorectal adenocarcinoma,NaN,11/29/2023,UNITED KINGDOM,ISTs,NaN,UNITED KINGDOM,1,AE,GBP,Europe,11,2023,4,1,2023-11-01,2023-11,NaN,2023Q4,Europe


### Commercial cases only

In [37]:
commercial_test  =new_df[new_df["Type"] == "Commercial"]
commercial_test.shape

(5838, 30)

In [38]:
commercial_test[commercial_test["Quarter_Year"]== "2023Q4"].shape

(392, 30)